In [7]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

# define CIFAR10 Dataset and load
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
batch_size = 4
Train_Dataset = torchvision.datasets.CIFAR10(root = './data',train = True,
                                            download=True, transform=transform)

Test_Dataset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                            download=True, transform=transform)

Train_loader = torch.utils.data.DataLoader(Train_Dataset, batch_size=batch_size,
                                           shuffle=True, num_workers=2)

Test_loader = torch.utils.data.DataLoader(Test_Dataset, batch_size = batch_size,
                                          shuffle=False, num_workers=2)

classes= ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')



Files already downloaded and verified
Files already downloaded and verified


In [8]:
#Define Convolutional Neural Network
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)


  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x= self.fc3(x)
    return x

net = Net()


In [11]:
#Lost Function and Optimizer
import torch.optim as optim
learning_rate = 0.001
momentum = 0.9
weight = 0.01
criterion = nn.CrossEntropyLoss()

# with regularization
optimizer = optim.SGD(net.parameters(),lr=learning_rate,momentum=momentum,weight_decay=weight)

In [12]:
#Training

# get some random training images
dataiter = iter(Train_loader)
images, labels = dataiter.next()

for epoch in range(2):
  loss = 0.0
  for i,data in enumerate(Train_loader,0):
    # get the inputs
    inputs,labels = data
    # zero the parameter gradients
    optimizer.zero_grad()

    outputs = net(inputs)
    loss_func = criterion(outputs,labels)
    loss_func.backward()
    optimizer.step()

    loss +=loss_func.item()
    if i % 1000 == 999:    # print every 1000 mini-batches
      print('[%d, %5d] loss: %.3f' %
      (epoch + 1, i + 1, loss / 2000))
      loss = 0.0


print('Training done !')

[1,  1000] loss: 0.698
[1,  2000] loss: 0.690
[1,  3000] loss: 0.690
[1,  4000] loss: 0.692
[1,  5000] loss: 0.684
[1,  6000] loss: 0.681
[1,  7000] loss: 0.705
[1,  8000] loss: 0.698
[1,  9000] loss: 0.685
[1, 10000] loss: 0.693
[1, 11000] loss: 0.685
[1, 12000] loss: 0.669
[2,  1000] loss: 0.663
[2,  2000] loss: 0.665
[2,  3000] loss: 0.667
[2,  4000] loss: 0.674
[2,  5000] loss: 0.651
[2,  6000] loss: 0.661
[2,  7000] loss: 0.656
[2,  8000] loss: 0.665
[2,  9000] loss: 0.658
[2, 10000] loss: 0.673
[2, 11000] loss: 0.666
[2, 12000] loss: 0.656
Training done !


In [14]:
#Accuracy
correct = 0
total = 0

with torch.no_grad():
  for data in Test_loader:
    images,labels = data
    outputs = net(images)

    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
accuracy = 100*(correct.item()/ total)

print('Accuracy of the network on the 10000 test images:',accuracy)

Accuracy of the network on the 10000 test images: 54.25


In [15]:
# prepare to count predictions for each class
correct_class = {classname: 0 for classname in classes}
total_class = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in Test_loader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)

        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_class[classes[label]] += 1
            total_class[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_class.items():
    accuracy_classes = 100 * float(correct_count) / total_class[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy_classes))

Accuracy for class plane is: 46.3 %
Accuracy for class car   is: 64.4 %
Accuracy for class bird  is: 58.4 %
Accuracy for class cat   is: 42.3 %
Accuracy for class deer  is: 49.0 %
Accuracy for class dog   is: 34.7 %
Accuracy for class frog  is: 69.4 %
Accuracy for class horse is: 52.2 %
Accuracy for class ship  is: 65.2 %
Accuracy for class truck is: 60.6 %
